In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',150)

In [11]:
def read_csv_with_progressbar(file_path, **kwargs):
    num_lines = sum(1 for line in open(file_path, encoding='latin-1')) 
    progress_bar = tqdm(total=num_lines)
    chunk_size = 10**6
    data_chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size, **kwargs):
        data_chunks.append(chunk)
        progress_bar.update(chunk_size)
    data = pd.concat(data_chunks)
    progress_bar.close()
    return data

In [9]:
%%time

cases = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/A_TblCase.csv', 
                    sep='\t', 
                    dtype='str', 
                    error_bad_lines=False, 
                    warn_bad_lines=False, 
                    encoding='latin-1', 
                    na_values=' ', 
                    quoting=csv.QUOTE_NONE)

proceedings = read_csv_with_progressbar("/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/B_TblProceeding.csv", 
                          sep='\t', 
                          dtype='str',
                          on_bad_lines='skip',
                          na_values=' ',
                          quoting=csv.QUOTE_NONE)

charges = read_csv_with_progressbar("/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/B_TblProceedCharges.csv", 
                      sep='\t',
                      dtype='str', 
                      na_values=' ',
                      on_bad_lines='skip',
                      quoting=csv.QUOTE_NONE)

charge_lookup = read_csv_with_progressbar("/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/Lookup/tbllookupCharges.csv", 
                            sep='\t',
                            dtype='str',
                            na_values=' ',
                            on_bad_lines='skip',
                            quoting=csv.QUOTE_NONE)
motions = read_csv_with_progressbar("/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/tbl_Court_Motions.csv", 
                            sep='\t',
                            dtype='str',
                            na_values=' ',
                            on_bad_lines='skip',
                            quoting=csv.QUOTE_NONE) 
schedule = read_csv_with_progressbar("/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/tbl_schedule.csv", 
                            sep='\t',
                            dtype='str',
                            na_values=' ',
                            on_bad_lines='skip',
                            quoting=csv.QUOTE_NONE)
case_identifier = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/A_TblCaseIdentifier.csv', 
                              sep='\t',
                              dtype='str',
                              na_values=' ',
                              on_bad_lines='skip',
                              quoting=csv.QUOTE_NONE)
schema = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/EOIRDB_Schema.csv', sep='\t',
                              dtype='str',
                              na_values=' ',
                              on_bad_lines='skip',
                              quoting=csv.QUOTE_NONE)

hlcodes = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/Lookup/tblLookupHloc.csv', sep='\t',
                              dtype='str',
                              na_values=' ',
                              on_bad_lines='skip',
                              quoting=csv.QUOTE_NONE)
case_id_lkup = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/Lookup/tblLookUpCaseIdentifier.csv', sep='\t',
                              dtype='str',
                              na_values=' ',
                              on_bad_lines='skip',
                              quoting=csv.QUOTE_NONE)
custody_history = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/tbl_CustodyHistory.csv', sep='\t',
                              dtype='str',
                              na_values=' ',
                              on_bad_lines='skip',
                              quoting=csv.QUOTE_NONE)
base_city_lkup = read_csv_with_progressbar('/Users/stjames/Dropbox/Pablo/data/EOIR TRAC JUNE/Lookup/tblLookupBaseCity.csv', sep='\t',
                              dtype='str',
                              na_values=' ',
                              on_bad_lines='skip',
                              quoting=csv.QUOTE_NONE)

  0%|          | 0/9360389 [00:00<?, ?it/s]<timed exec>:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


<timed exec>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


10000000it [00:40, 247868.42it/s]                            
13000000it [01:13, 176936.86it/s]                              
15000000it [00:18, 813727.73it/s]                              
1000000it [00:00, 42894877.33it/s]     
5000000it [01:12, 68746.31it/s]                              
34000000it [04:28, 126530.70it/s]                              
2000000it [00:01, 1166052.27it/s]                            
1000000it [00:00, 89196861.11it/s]      
1000000it [00:00, 32323800.28it/s]     
1000000it [00:00, 126567005.64it/s]    
7000000it [00:06, 1014616.25it/s]                            
1000000it [00:00, 74361818.31it/s]    


CPU times: user 5min 52s, sys: 2min 4s, total: 7min 56s
Wall time: 9min 24s


In [12]:
# clean motions table

motions = motions.reset_index()

# Shift column names two spaces to the left

motions.columns = list(motions.columns[2:]) + ['blank 1', 'blank 2']

In [84]:
# clean schedule & proceedings table

# Function to check if a value is numerical
def is_numerical(value):
    if isinstance(value, (int, float)):
        return True
    elif isinstance(value, str) and value.isdigit():
        return True
    return False

# Vectorize the function to work with Pandas Series
vectorized_is_numerical = np.vectorize(is_numerical)

# Create a boolean mask indicating valid IDNCASE values
valid_idncase_mask = vectorized_is_numerical(schedule["IDNCASE"])
valid_idncase_mask_1 = vectorized_is_numerical(proceedings["IDNCASE"])


# Print the invalid IDNCASE values
print("Dropping rows with non-numerical IDNCASE values:")
print(schedule.loc[~valid_idncase_mask, "IDNCASE"])
print(proceedings.loc[~valid_idncase_mask_1, "IDNCASE"])


# Use boolean indexing to keep only rows with numerical IDNCASE values
schedule = schedule[valid_idncase_mask]
proceedings = proceedings[valid_idncase_mask_1]


# convert IDNCASE to int if they are not NA and are digits
proceedings['IDNCASE'] = pd.to_numeric(proceedings['IDNCASE'], errors='coerce').dropna().astype(int)
schedule['IDNCASE'] = pd.to_numeric(schedule['IDNCASE'], errors='coerce').dropna().astype(int)


# convert date values to dt

schedule.INPUT_DATE = pd.to_datetime(schedule.INPUT_DATE, errors='coerce')

Dropping rows with non-numerical IDNCASE values:
Series([], Name: IDNCASE, dtype: int64)
Series([], Name: IDNCASE, dtype: int64)


In [15]:
# convert IDNCASE to int if they are not NA and are digits
case_identifier['IDNCASE'] = pd.to_numeric(case_identifier['IDNCASE'], errors='coerce').dropna().astype(int)
cases['IDNCASE'] = pd.to_numeric(cases['IDNCASE'], errors='coerce').dropna().astype(int)

In [16]:
print(f"Cases: {cases.shape[0]:,}")
print(f"Proceedings: {proceedings.shape[0]:,}")
print(f"Motions: {motions.shape[0]:,}")
print(f"Schedule: {schedule.shape[0]:,}")

Cases: 9,360,337
Proceedings: 12,148,766
Motions: 4,629,241
Schedule: 33,813,940


In [17]:
cases.head()

,IDNCASE,ALIEN_CITY,ALIEN_STATE,ALIEN_ZIPCODE,UPDATED_ZIPCODE,UPDATED_CITY,NAT,LANG,CUSTODY,SITE_TYPE,E_28_DATE,ATTY_NBR,CASE_TYPE,UPDATE_SITE,LATEST_HEARING,LATEST_TIME,LATEST_CAL_TYPE,UP_BOND_DATE,UP_BOND_RSN,CORRECTIONAL_FAC,RELEASE_MONTH,RELEASE_YEAR,INMATE_HOUSING,DATE_OF_ENTRY,C_ASY_TYPE,C_BIRTHDATE,C_RELEASE_DATE,UPDATED_STATE,ADDRESS_CHANGEDON,ZBOND_MRG_FLAG,GENDER,DATE_DETAINED,DATE_RELEASED,LPR,DETENTION_DATE,DETENTION_LOCATION,DCO_LOCATION,DETENTION_FACILITY_TYPE,CASEPRIORITY_CODE
0,11812380,OAKLAND,CA,94601,NaN,NaN,ES,SP,N,M,NaN,NaN,RMV,SFR,2025-05-27 00:00:00.000,1330,M,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-14 00:00:00.000,NaN,11/2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11812381,CARROLTON,TX,75007,NaN,NaN,PE,SP,N,M,NaN,NaN,RMV,DAL,2027-04-07 00:00:00.000,1300,M,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-17 00:00:00.000,NaN,10/1998,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11812382,NaN,NaN,NaN,NaN,NaN,ER,ENG,N,M,NaN,NaN,RMV,NYC,2023-11-28 00:00:00.000,1030,M,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-03 00:00:00.000,NaN,3/1992,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11812383,SARASOTA,FL,34234,NaN,NaN,VE,SP,N,M,NaN,NaN,RMV,ORL,2025-07-15 00:00:00.000,0830,M,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-03 00:00:00.000,NaN,7/1995,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11812384,SARASOTA,FL,34234,NaN,NaN,VE,SP,N,M,NaN,NaN,RMV,ORL,2025-07-15 00:00:00.000,0830,M,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-03 00:00:00.000,NaN,2/2016,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
proceedings.head()

,IDNPROCEEDING,IDNCASE,OSC_DATE,INPUT_DATE,BASE_CITY_CODE,HEARING_LOC_CODE,IJ_CODE,TRANS_IN_DATE,PREV_HEARING_LOC,PREV_HEARING_BASE,PREV_IJ_CODE,TRANS_NBR,HEARING_DATE,HEARING_TIME,DEC_TYPE,DEC_CODE,DEPORTED_1,DEPORTED_2,OTHER_COMP,APPEAL_RSVD,APPEAL_NOT_FILED,COMP_DATE,ABSENTIA,VENUE_CHG_GRANTED,TRANSFER_TO,DATE_APPEAL_DUE_STATUS,TRANSFER_STATUS,CUSTODY,CASE_TYPE,NAT,LANG,SCHEDULED_HEAR_LOC,CORRECTIONAL_FAC,CRIM_IND,IHP,AGGRAVATE_FELON,DATE_DETAINED,DATE_RELEASED
0,10661434,10095854,2021-11-23 00:00:00.000,2022-03-29 00:00:00.000,PHI,PHI,JHS,2022-04-05 00:00:00.000,NAT,NEW,PLE,01,2023-04-12 00:00:00.000,1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,N,RMV,BR,POR,NaN,NaN,N,NaN,0,NaN,NaN
1,10661435,10504057,2022-03-30 00:00:00.000,2022-03-30 00:00:00.000,HSG,HSG,NES,NaN,NaN,NaN,NaN,NaN,2022-12-05 00:00:00.000,0830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,RMV,CU,SP,NaN,NaN,N,NaN,0,NaN,NaN
2,10661437,9589112,2019-05-11 00:00:00.000,2022-03-30 00:00:00.000,NYV,NYV,TSM,2022-04-04 00:00:00.000,WAS,WAS,LL1,01,2023-07-11 00:00:00.000,0830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,N,RMV,HO,SP,NaN,NaN,N,NaN,0,NaN,NaN
3,10661438,10504059,2022-03-30 00:00:00.000,2022-03-30 00:00:00.000,SMO,SMO,LSG,NaN,NaN,NaN,NaN,NaN,2022-05-16 00:00:00.000,0800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,RMV,RU,RUS,NaN,NaN,N,NaN,0,NaN,NaN
4,10661440,10504060,2022-03-30 00:00:00.000,2022-04-28 00:00:00.000,SNA,SNA,DJS,NaN,NaN,NaN,NaN,NaN,2023-04-20 00:00:00.000,0900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,RMV,HO,SP,NaN,NaN,N,NaN,0,NaN,NaN


In [19]:
schedule.head()

,IDNSCHEDULE,IDNPROCEEDING,IDNCASE,OSC_DATE,GENERATION,SUB_GENERATION,REC_TYPE,LANG,HEARING_LOC_CODE,BASE_CITY_CODE,IJ_CODE,INTERPRETER_CODE,INPUT_DATE,INPUT_TIME,UPDATE_DATE,UPDATE_TIME,ASSIGNMENT_PATH,CAL_TYPE,ADJ_DATE,ADJ_TIME_START,ADJ_TIME_STOP,ADJ_RSN,ADJ_MEDIUM,ADJ_MSG,ADJ_ELAP_DAYS,LNGSESSNID,SCHEDULE_TYPE,NOTICE_CODE,BLNCLOCKOVERRIDE,EOIRAttorneyID
0,30032802,7568117,8145563,2016-08-12 00:00:00.000,99,NaN,X,SP,LOS,LOS,ARD,NaN,2017-03-11,1528,2017-06-19 17:10:48.000,1710,NaN,M,2017-06-19 00:00:00.000,1330,1400,01,P,NaN,0000,8135098,MM,T3,0,NaN
1,30032805,7568160,8145606,2016-05-06 00:00:00.000,99,NaN,X,SP,LOS,LOS,ARD,NaN,2017-03-11,1528,2022-05-12 14:17:29.000,1417,NaN,M,2017-06-12 00:00:00.000,1330,1400,21,P,NaN,0053,8135086,MM,T3,0,NaN
2,30032806,7568159,8145605,2016-07-15 00:00:00.000,99,NaN,X,SP,LOS,LOS,HO,NaN,2017-03-11,1529,2017-05-23 14:16:12.000,1416,NaN,M,2017-06-13 00:00:00.000,1330,1400,35,P,NaN,0000,8135092,MM,N6,0,NaN
3,30032807,7568161,8145607,2016-08-24 00:00:00.000,99,NaN,X,SP,LOS,LOS,HO,NaN,2017-03-11,1530,2017-05-31 12:04:27.000,1204,NaN,M,2017-06-19 00:00:00.000,1330,1400,14,P,NaN,0000,8135098,MM,T3,0,NaN
4,30032808,7568164,8145610,2016-08-24 00:00:00.000,99,NaN,X,SP,LOS,LOS,HO,NaN,2017-03-11,1532,2017-05-31 12:04:27.000,1204,NaN,M,2017-06-19 00:00:00.000,1330,1400,14,P,NaN,0000,8135098,MM,T3,0,NaN


In [20]:
# filtering tables
# demo that label CASETYPE = Franco Litigation is not a good marker in case identifier table
# demo that not all cases that receive QR also received JCI or are labeled Franco Litigation

In [21]:
# the adjournment code for 'Appointed Qualified Representative' is 61, cross checked on the tblAdjournmentCodes LookUP
# "Adjourned to allow for the appointment of a qualified representative for the alien"
# the adjournment code for 'Judicial Competency Inquiry' is 61, cross checked on the tblAdjournmentCodes LookUP
# "Adjourned and set for a judicial competency inquiry at a subsequent hearing."

In [22]:
# creating filters

# selecting all of the cases labeled as franco litigation
FL = case_identifier[case_identifier['CASE_ID']=='FL']['IDNCASE'].drop_duplicates().to_frame()

# all cases

JCI = schedule[schedule['ADJ_RSN']=='62']['IDNCASE'].drop_duplicates().to_frame()
QR = schedule[schedule['ADJ_RSN']=='61']['IDNCASE'].drop_duplicates().to_frame()

# all proceedings

JCI_P = schedule[schedule['ADJ_RSN']=='62']['IDNPROCEEDING'].drop_duplicates().to_frame()
QR_P = schedule[schedule['ADJ_RSN']=='61']['IDNPROCEEDING'].drop_duplicates().to_frame()

# any case that received either a JCI, a QR or both
JCI_QR = combined = pd.concat([JCI, QR]).drop_duplicates()

# proceeding

JCI_QR_P = combined = pd.concat([JCI_P, QR_P]).drop_duplicates()

# filtering cases labeled Franco Litigation

FL_QR = schedule[(schedule['IDNCASE'].isin(FL['IDNCASE'])) & (schedule['IDNCASE'].isin(QR['IDNCASE']))]['IDNCASE'].drop_duplicates().to_frame()
FL_JCI = schedule[(schedule['IDNCASE'].isin(FL['IDNCASE'])) & (schedule['IDNCASE'].isin(JCI['IDNCASE']))]['IDNCASE'].drop_duplicates().to_frame()


In [23]:
# select opposite of FL IDNCASE IDs from schedule table for comparison

NOT_FL = schedule[~schedule['IDNCASE'].isin(FL['IDNCASE'])]['IDNCASE'].drop_duplicates().to_frame()

NOT_FL_QR = NOT_FL[NOT_FL['IDNCASE'].isin(QR['IDNCASE'])]
NOT_FL_JCI = NOT_FL[NOT_FL['IDNCASE'].isin(JCI['IDNCASE'])]

In [24]:
print(f"# of cases labeled Franco Litigation: {FL.IDNCASE.nunique():,}")
print(f"# of cases labeled Franco Litigation and received at least 1 JCI: {FL_JCI.IDNCASE.nunique():,}")
print(f"# of cases labeled Franco Litigation and appointed QR: {FL_QR.IDNCASE.nunique():,}")


# of cases labeled Franco Litigation: 1,868
# of cases labeled Franco Litigation and received at least 1 JCI: 1,103
# of cases labeled Franco Litigation and appointed QR: 836


In [25]:
print(f"# of cases not labeled Franco Litigation: {NOT_FL.IDNCASE.nunique():,}")
print(f"# of cases not labeled Franco Litigation and received at least 1 JCI: {NOT_FL_JCI.IDNCASE.nunique():,}")
print(f"# of cases not labeled Franco Litigation and appointed QR: {NOT_FL_QR.IDNCASE.nunique():,}")

# of cases not labeled Franco Litigation: 9,155,395
# of cases not labeled Franco Litigation and received at least 1 JCI: 2,822
# of cases not labeled Franco Litigation and appointed QR: 1,316


In [26]:
print(f"# of all cases: {schedule.IDNCASE.nunique():,}")
print(f"# of all cases received at least 1 JCI: {JCI.IDNCASE.nunique():,}")
print(f"# of all cases appointed QR: {QR.IDNCASE.nunique():,}")

# of all cases: 9,157,248
# of all cases received at least 1 JCI: 3,925
# of all cases appointed QR: 2,152


In [27]:
# demo that not all w/ FL labeled cases QR received JCI
common_values = FL_QR.IDNCASE.drop_duplicates().isin(FL_JCI.IDNCASE.drop_duplicates()).value_counts()

print("Number of Franco Litigation labled cases appointed QR:", common_values[0]+common_values[1])
print("Number of Franco Litigation labled cases appointed QR who also received JCI:", common_values[1])

Number of Franco Litigation labled cases appointed QR: 836
Number of Franco Litigation labled cases appointed QR who also received JCI: 532


In [28]:
# same issue where not all cases w/ QR receive JCI persists in cases not labeled Franco Litigation

common_values = NOT_FL_QR.IDNCASE.drop_duplicates().isin(NOT_FL_JCI.IDNCASE.drop_duplicates()).value_counts()

print("Number of IDNCASE not labeled Franco Litigation and appointed QR:", common_values[0]+common_values[1])
print("Number of IDNCASE not labeled Franco Litigation appointed QR who also received JCI:", common_values[1])

Number of IDNCASE not labeled Franco Litigation and appointed QR: 1316
Number of IDNCASE not labeled Franco Litigation appointed QR who also received JCI: 604


In [29]:
# finally, demonstrating this for the entire schedule - cases labeled FL and not labeled FL

common_values = QR.IDNCASE.drop_duplicates().isin(JCI.IDNCASE.drop_duplicates()).value_counts()

print("Number of cases appointed QR:", common_values[0]+common_values[1])
print("Number of cases appointed QR who also received JCI:", common_values[1])

Number of cases appointed QR: 2152
Number of cases appointed QR who also received JCI: 1136


In [30]:
# replicating this w/ proceedings
# this is a more accurate representation because this shows a direct progression between JCI and QR 

common_values = QR_P.IDNPROCEEDING.drop_duplicates().isin(JCI_P.IDNPROCEEDING.drop_duplicates()).value_counts()

print("Number of proceedings appointed QR:", common_values[0]+common_values[1])
print("Number of proceedings appointed QR which also received JCI:", common_values[1])

Number of proceedings appointed QR: 2174
Number of proceedings appointed QR which also received JCI: 1060


In [31]:
# saving this subset of cases that received QR without JCI for future analysis

mask = ~QR.IDNCASE.drop_duplicates().isin(JCI.IDNCASE.drop_duplicates())
NO_JCI_QR = QR.IDNCASE.drop_duplicates()[mask]
NO_JCI_QR = NO_JCI_QR.to_frame()
NO_JCI_QR.IDNCASE = NO_JCI_QR.IDNCASE.astype('int')

In [32]:
# another issue is that there are cases that received QR or JCI that do not appear in the case identifier table at all

In [33]:
CI_JCI_QR = case_identifier[case_identifier['IDNCASE'].isin(JCI_QR.IDNCASE)]

print("Number of cases received either a JCI, a QR or both:", JCI_QR.IDNCASE.nunique())
print("Number of these cases that appear in the case identifier table:", CI_JCI_QR.IDNCASE.nunique())
print("Number of these cases labled Franco litigation:", CI_JCI_QR[CI_JCI_QR['IDNCASE'].isin(FL['IDNCASE'])].IDNCASE.nunique())
print("Number of these cases *not* labled Franco litigation:", CI_JCI_QR[~CI_JCI_QR['IDNCASE'].isin(FL['IDNCASE'])].IDNCASE.nunique())

Number of cases received either a JCI, a QR or both: 4941
Number of these cases that appear in the case identifier table: 1832
Number of these cases labled Franco litigation: 1407
Number of these cases *not* labled Franco litigation: 425


In [34]:
# this may just be a feature of the EOIR data, because this appears to happen across the board for all cases.

In [35]:
print("Number of JCI/QR cases that do not appear in the case identifier table:", JCI_QR.IDNCASE.nunique() - CI_JCI_QR.IDNCASE.nunique())
print("Number of all cases that do not appear in the case identifier table:", cases.IDNCASE.nunique() - case_identifier.IDNCASE.nunique())

case_identifier.IDNCASE.nunique()

Number of JCI/QR cases that do not appear in the case identifier table: 3109
Number of all cases that do not appear in the case identifier table: 8051800


1308537

In [36]:
# of JCI/QR cases that *do* appear in the case_identifier table, these are how they are labeled

In [37]:
# decoding the case_identifier table

In [38]:
# rename the strcode column in df_lookup to CASE_ID to match df_coded
case_id_lkup.rename(columns={'strcode': 'CASE_ID'}, inplace=True)

# merge the two DataFrames on the 'CASE_ID' column
CI_JCI_QR = pd.merge(CI_JCI_QR, case_id_lkup, on='CASE_ID', how='left')

# drop extra columns

CI_JCI_QR = CI_JCI_QR[['IDNCASEID','IDNCASE', 'CASE_ID','idnCaseIdentifier','strDescription']]

In [39]:
# case ID labels for cases that received FL label at least once
# note that IDNCASE is often duplicated in this table and labled as various types of cases

In [40]:
CI_JCI_QR[CI_JCI_QR['IDNCASE'].isin(FL['IDNCASE'])].strDescription.value_counts()

Franco Litigation                                  1412
Civil Detention Hearing                             270
OFF CALENDAR INITIATIVE                             128
CaseFlow Management Process                          32
Custody Review                                       23
Port Court 2 - Otay Mesa                             20
PRE-TRIAL/HEARING                                    20
Reset Order Work Flow                                10
Status Docket                                         9
Pulled Case                                           6
Reserved Suspension Decision                          5
Vertical Prosecution                                  5
IHP case where NTA was filed within 120 of EPRD       4
ATD Pilot                                             2
MIGRANT PROTECTION PROTOCOL                           2
INTERIM FINAL RULE                                    1
MENDEZ-ROJAS CASE                                     1
Dedicated Docket                                

In [41]:
# case ID labels for cases that never received FL label
# note that IDNCASE is often duplicated in this table and labled as various types of cases

In [42]:
CI_JCI_QR[~CI_JCI_QR['IDNCASE'].isin(FL['IDNCASE'])].strDescription.value_counts()

CaseFlow Management Process                        115
Civil Detention Hearing                             51
Pulled Case                                         41
OFF CALENDAR INITIATIVE                             35
Family Units                                        34
Status Docket                                       32
Reset Order Work Flow                               28
Dedicated Docket                                    26
Reserved Suspension Decision                        17
Port Court 2 - Otay Mesa                            13
PRE-TRIAL/HEARING                                   11
MIGRANT PROTECTION PROTOCOL                         11
The Brito Class Action Litigation                    9
IHP case where NTA was filed within 120 of EPRD      6
MENDEZ-ROJAS CASE                                    6
Custody Review                                       5
Padilla Class Members                                5
NACARA Section 203                                   4
ATD Pilot 

In [43]:
# FL label not helpful

# given these anomalies, will proceed w/ 5 categories for now

# cases that received a QR or JCI. largest category. (JCI_QR) 
# cases that received a JCI (JCI)
# cases that received a QR. Assuming JCI occured, but not recorded for anyone receiving a QR w/ no JCI. (QR)
# cases that received a JCI and QR. more precise than QR, if assumptions are wrong. (JCI_AND_QR)

JCI_AND_QR = QR[QR['IDNCASE'].isin(JCI['IDNCASE'])]['IDNCASE'].drop_duplicates().to_frame()
JCI_AND_QR_P = QR_P[QR_P['IDNPROCEEDING'].isin(JCI_P['IDNPROCEEDING'])]['IDNPROCEEDING'].drop_duplicates().to_frame()

# can easily expand these categories if necessary 

In [44]:
# address issue of cases where QR was appointed in a different proceeding to that of the JCI. not quite sure if this is enough to put them into a fully different category, but will add a 5th category in case

In [45]:
JCI_AND_QR_CLEANED = schedule[schedule.IDNPROCEEDING.isin(JCI_AND_QR_P.IDNPROCEEDING)].IDNCASE.drop_duplicates().to_frame()

In [46]:
A = schedule[schedule.IDNPROCEEDING.isin(JCI_AND_QR_P.IDNPROCEEDING)].IDNCASE.drop_duplicates().to_frame() # certain there is a QR outcome for the JCI

In [47]:
B = schedule[schedule.IDNCASE.isin(JCI_AND_QR.IDNCASE)].IDNCASE.drop_duplicates().to_frame() # for many there is a QR outcome for the tallied QR, but for others its unclear

In [48]:
C = B[~B.IDNCASE.isin(A.IDNCASE)] # this is the group of cases where a QR was appointed in a different proceeding from the JCI

In [49]:
# still, there are only a small number of cases  like this. 

In [50]:
C.IDNCASE.count() 

76

In [51]:
# these are some example of the above cases on the schedule table

In [52]:
schedule[schedule.IDNCASE==8222761].sort_index()

,IDNSCHEDULE,IDNPROCEEDING,IDNCASE,OSC_DATE,GENERATION,SUB_GENERATION,REC_TYPE,LANG,HEARING_LOC_CODE,BASE_CITY_CODE,IJ_CODE,INTERPRETER_CODE,INPUT_DATE,INPUT_TIME,UPDATE_DATE,UPDATE_TIME,ASSIGNMENT_PATH,CAL_TYPE,ADJ_DATE,ADJ_TIME_START,ADJ_TIME_STOP,ADJ_RSN,ADJ_MEDIUM,ADJ_MSG,ADJ_ELAP_DAYS,LNGSESSNID,SCHEDULE_TYPE,NOTICE_CODE,BLNCLOCKOVERRIDE,EOIRAttorneyID
2332853,31578834,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,X,SP,SFD,SFR,JYP,NaN,2018-02-22,1252,2018-03-09 15:49:13.000,1549,NaN,I,2018-03-22 00:00:00.000,0900,0930,12,P,NaN,0000,9193733,ID,,0,NaN
2354254,31579826,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,B,SP,SFD,SFR,JYP,NaN,2018-02-22,1417,2018-02-22 14:20:19.000,1420,NaN,I,2018-03-01 00:00:00.000,1300,1330,99,P,NaN,0000,9193633,CY,W7,0,NaN
2354289,31579864,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,B,SP,SFD,SFR,JYP,NaN,2018-02-22,1420,2018-03-09 15:48:54.000,1548,NaN,I,2018-03-09 00:00:00.000,0900,0930,8A,P,NaN,0000,9193671,CY,W7,0,NaN
3441078,31656766,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,X,SP,SFD,SFR,JYP,NaN,2018-03-09,1549,2018-04-26 14:16:10.000,1416,NaN,I,2018-04-26 00:00:00.000,0900,0930,12,P,NaN,0000,9214898,ID,,0,B61DAC9A-1172-4AD2-9C50-41FAF96CA394
3865720,31654636,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,X,SP,SFD,SFR,JYP,NaN,2018-03-09,1138,2018-03-09 15:36:09.000,1536,NaN,M,2018-03-09 00:00:00.000,0900,0930,8A,V,NaN,0000,9193671,MM,,0,B61DAC9A-1172-4AD2-9C50-41FAF96CA394
3905110,31689391,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,B,SP,SFD,SFR,JYP,NaN,2018-03-16,1417,2018-04-26 14:16:20.000,1416,NaN,I,2018-04-26 00:00:00.000,0900,0930,12,P,NaN,0000,9214898,CY,,0,B61DAC9A-1172-4AD2-9C50-41FAF96CA394
6663476,31884453,7879357,8222761,2017-05-22 00:00:00.000,96,NaN,X,SP,SFD,SFR,JYP,NaN,2018-04-26,1416,2019-09-05 16:40:38.980,1640,NaN,I,2018-08-10 00:00:00.000,1300,1700,RR,P,NaN,0046,9545833,ID,,0,B61DAC9A-1172-4AD2-9C50-41FAF96CA394
7752634,34320152,8744904,8222761,2017-05-22 00:00:00.000,95,NaN,X,SP,SFD,SFR,CSG,NaN,2019-06-11,1118,2019-06-14 18:00:05.000,1800,NaN,M,2019-06-14 00:00:00.000,0900,0930,17,V,NaN,0000,10974527,MM,N3,0,B61DAC9A-1172-4AD2-9C50-41FAF96CA394
8322897,34359386,8744904,8222761,2017-05-22 00:00:00.000,95,NaN,X,SP,SFD,SFR,JYP,NaN,2019-06-14,1800,2019-08-22 10:42:52.000,1042,NaN,I,2019-08-08 00:00:00.000,0830,1130,RR,P,NaN,0000,11240761,ID,N3,0,B61DAC9A-1172-4AD2-9C50-41FAF96CA394
19539710,30416672,7678333,8222761,2017-05-22 00:00:00.000,99,NaN,X,SP,SFV,SFR,JLN,NaN,2017-05-31,1708,2017-06-12 13:45:07.000,1345,NaN,M,2017-06-08 00:00:00.000,1300,1330,62,V,NaN,0000,8559534,MM,VW,0,NaN


In [53]:
schedule[schedule.IDNCASE==8432612].sort_index()

,IDNSCHEDULE,IDNPROCEEDING,IDNCASE,OSC_DATE,GENERATION,SUB_GENERATION,REC_TYPE,LANG,HEARING_LOC_CODE,BASE_CITY_CODE,IJ_CODE,INTERPRETER_CODE,INPUT_DATE,INPUT_TIME,UPDATE_DATE,UPDATE_TIME,ASSIGNMENT_PATH,CAL_TYPE,ADJ_DATE,ADJ_TIME_START,ADJ_TIME_STOP,ADJ_RSN,ADJ_MEDIUM,ADJ_MSG,ADJ_ELAP_DAYS,LNGSESSNID,SCHEDULE_TYPE,NOTICE_CODE,BLNCLOCKOVERRIDE,EOIRAttorneyID
1790160,36599852,8321052,8432612,2018-02-07 00:00:00.000,95,NaN,X,SP,PDC,DAL,EMB,NaN,2020-04-08,1346,2020-06-01 14:59:09.630,1459,NaN,I,2020-04-17 00:00:00.000,0830,1130,12,V,NaN,0009,12633424,ID,VW,0,798949FB-4400-4307-80DB-046C7E35B5D8
2205182,31569142,8002704,8432612,2018-02-07 00:00:00.000,99,NaN,X,SP,PDC,DAL,VPG,NaN,2018-02-21,1119,2018-03-06 11:21:31.000,1121,NaN,M,2018-03-06 00:00:00.000,0830,0900,60,V,NaN,0000,9138671,MM,Y2,0,NaN
2263129,36636981,8321052,8432612,2018-02-07 00:00:00.000,95,NaN,X,SP,PDC,DAL,JN,NaN,2020-04-14,1514,2020-06-01 14:59:09.630,1459,NaN,I,2020-06-01 00:00:00.000,1300,1700,8A,H,NaN,0000,8839378,ID,VW,0,798949FB-4400-4307-80DB-046C7E35B5D8
3113728,31632548,8002704,8432612,2018-02-07 00:00:00.000,99,NaN,X,SP,PDC,DAL,JN,NaN,2018-03-06,1124,2018-04-23 15:32:56.000,1532,NaN,M,2018-04-10 00:00:00.000,0900,1000,62,V,NaN,0000,8172363,MR,N6,0,NaN
3421914,34022665,8321052,8432612,2018-02-07 00:00:00.000,95,NaN,X,SP,PDC,DAL,JN,NaN,2019-05-02,1018,2019-06-04 10:49:41.000,1049,NaN,M,2019-06-04 00:00:00.000,0900,1000,02,V,NaN,0000,10968531,MR,Y2,0,798949FB-4400-4307-80DB-046C7E35B5D8
6331294,31860638,8068294,8432612,2018-02-07 00:00:00.000,98,NaN,X,SP,DAD,DAL,JN,NaN,2018-04-23,1533,2018-05-07 14:06:40.000,1406,NaN,I,2018-05-04 00:00:00.000,1000,1200,02,P,NaN,0000,9621729,II,V3,0,NaN
6983242,34268227,8321052,8432612,2018-02-07 00:00:00.000,95,NaN,X,SP,PDC,DAL,JN,NaN,2019-06-04,1050,2019-07-02 10:59:33.000,1059,NaN,M,2019-07-02 00:00:00.000,0900,0930,02,V,NaN,0000,11060034,MR,N6,0,798949FB-4400-4307-80DB-046C7E35B5D8
7355218,31931737,8104743,8432612,2018-02-07 00:00:00.000,97,NaN,X,SP,PDC,DAL,JN,NaN,2018-05-07,1407,2018-05-21 12:19:00.000,1219,NaN,M,2018-05-21 00:00:00.000,0900,1000,01,V,NaN,0000,8202670,MR,Y2,0,NaN
8403822,32006642,8104743,8432612,2018-02-07 00:00:00.000,97,NaN,X,SP,PDC,DAL,JN,NaN,2018-05-21,1219,2018-06-19 11:09:11.000,1109,NaN,M,2018-06-19 00:00:00.000,0900,1000,54,V,NaN,0000,9433524,MR,Y2,0,NaN
9898213,34468919,8321052,8432612,2018-02-07 00:00:00.000,95,NaN,X,SP,PDC,DAL,JN,NaN,2019-07-02,1059,2019-07-25 11:52:56.000,1152,NaN,M,2019-07-25 00:00:00.000,0900,0930,02,V,NaN,0000,11129343,MR,N6,0,798949FB-4400-4307-80DB-046C7E35B5D8


In [54]:
CATEGORIES = {'JCI_QR':JCI_QR, 'JCI':JCI, 'QR':QR, 'JCI_AND_QR':JCI_AND_QR, 'JCI_AND_QR_CLEANED':JCI_AND_QR_CLEANED}

PROCEEDING_CATEGORIES = {'JCI_QR_P':JCI_QR_P, 'JCI_P':JCI_P, 'QR_P':QR_P, 'JCI_AND_QR_P':JCI_AND_QR_P}

for name, df in CATEGORIES.items():
    print(f"Category Name: {name}, Number of cases: {df.shape[0]}")

for name, df in PROCEEDING_CATEGORIES.items():
    print(f"Proceeding Category Name: {name}, Number of proceedings: {df.shape[0]}")

Category Name: JCI_QR, Number of cases: 4941
Category Name: JCI, Number of cases: 3925
Category Name: QR, Number of cases: 2152
Category Name: JCI_AND_QR, Number of cases: 1136
Category Name: JCI_AND_QR_CLEANED, Number of cases: 1061
Proceeding Category Name: JCI_QR_P, Number of proceedings: 5128
Proceeding Category Name: JCI_P, Number of proceedings: 4014
Proceeding Category Name: QR_P, Number of proceedings: 2174
Proceeding Category Name: JCI_AND_QR_P, Number of proceedings: 1060


In [55]:
# filter detained cases
# can filter detained cases either in the A_TblCase DataFrame or the custody history DataFrame

In [56]:
# detained cases. there are duplicates because people may have been detained more than once. 
DET = custody_history[custody_history['CUSTODY']!='N']

In [57]:
NON_DET = JCI_QR[~JCI_QR['IDNCASE'].astype('int').isin(DET['IDNCASE'].astype('int'))]

In [58]:
def analyze_category(category_df, category_name):
    # detained cases. there are duplicates because people may have been detained more than once
    DET = custody_history[custody_history['CUSTODY']!='N']
    DET = DET[DET['IDNCASE'].astype('int').isin(category_df['IDNCASE'].astype('int'))]
    NON_DET = category_df[~category_df['IDNCASE'].astype('int').isin(DET['IDNCASE'].astype('int'))]

    return (category_df.shape[0], NON_DET.shape[0], cases[(cases['IDNCASE'].isin(category_df['IDNCASE']))].IDNCASE.nunique(), cases[(cases['IDNCASE'].isin(category_df['IDNCASE'])) & (cases['CUSTODY']=='N')]['IDNCASE'].nunique())


CustodyHistory_results = []
ATblCase_results = []

for name, df in CATEGORIES.items():
    total_cases, non_det_cases, cases_in_name, never_det_cases = analyze_category(df, name)
    CustodyHistory_results.append(f"For {name}, out of {total_cases} cases, {non_det_cases} were labeled as never detained.")
    ATblCase_results.append(f"For {name}, out of {cases_in_name} cases, {never_det_cases} were labeled as never detained.")

print("Results according to tbl_CustodyHistory.CSV:")
for result in CustodyHistory_results:
    print(result)

print("\nResults according to A_TblCase.CSV:")
for result in ATblCase_results:
    print(result)


Results according to tbl_CustodyHistory.CSV:
For JCI_QR, out of 4941 cases, 522 were labeled as never detained.
For JCI, out of 3925 cases, 476 were labeled as never detained.
For QR, out of 2152 cases, 49 were labeled as never detained.
For JCI_AND_QR, out of 1136 cases, 3 were labeled as never detained.
For JCI_AND_QR_CLEANED, out of 1061 cases, 3 were labeled as never detained.

Results according to A_TblCase.CSV:
For JCI_QR, out of 4941 cases, 516 were labeled as never detained.
For JCI, out of 3925 cases, 472 were labeled as never detained.
For QR, out of 2152 cases, 45 were labeled as never detained.
For JCI_AND_QR, out of 1136 cases, 1 were labeled as never detained.
For JCI_AND_QR_CLEANED, out of 1061 cases, 1 were labeled as never detained.


In [59]:
# this make the JCI_AND_QR_CLEANED, similar to JCI__AND_QR_P category, look better

In [60]:
# filter california
# probably need to use BASE_CITY_CODE

In [61]:
CA = base_city_lkup[base_city_lkup['BASE_STATE']=='CA']

In [62]:
# here are the top 5 locations where our target proceedings have occured

In [63]:
pd.merge(proceedings[proceedings.IDNPROCEEDING.isin(JCI_AND_QR_P.IDNPROCEEDING)], base_city_lkup, on='BASE_CITY_CODE', how='left').BASE_STATE.value_counts().head()

CA    493
AZ    175
WA    112
CO     53
VA     50
Name: BASE_STATE, dtype: int64

In [64]:
# i want to account for transfers. i dont want to include proceedings that started in one location and ended in another. 

In [65]:
D = pd.merge(proceedings[proceedings.IDNCASE.isin(JCI_AND_QR_P.IDNPROCEEDING)], base_city_lkup, on='BASE_CITY_CODE', how='left')

In [66]:
# Ensure the DataFrame is sorted by IDNPROCEEDING
D = D.sort_values('IDNPROCEEDING')

# Create a new column, 'previous_state', that gives the BASE_STATE value from the previous row
D['previous_state'] = D.groupby('IDNCASE')['BASE_STATE'].shift()

# Create a new column, 'state_change', that indicates where BASE_STATE changes from 'CA' to something else
D['state_change'] = (D['previous_state'] == 'CA') & (D['BASE_STATE'] != 'CA')

# Now, filter to rows where state_change is True
D_state_change = D[D['state_change']]


In [67]:
# i've tried the above function for the various selections i've made, and JCI_AND_QR_P is the only one with no transfers away from CA. 
# I can ensure that all proceedings happen in same state w/ this filter.

D_state_change.shape

(0, 137)

In [68]:
# to recap, with the filter JCI_AND_QR_P, i can ensure the following:
# 1) QR occurs with a JCI
# 2) QR appointed in the same proceeding as JCI
# 3) JCI and QR occur in the same state

# this filter identifies 493 proceedings / cases matching critieria 

In [69]:
# i will attempt to create a different filter to see if i missed anything

In [70]:
JCI_P_CA = schedule[(schedule['ADJ_RSN']=='62') & (schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNPROCEEDING'].drop_duplicates().to_frame()
QR_P_CA = schedule[(schedule['ADJ_RSN']=='61') & (schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNPROCEEDING'].drop_duplicates().to_frame()

In [71]:
JCI_P_CA.shape

(1549, 1)

In [72]:
QR_P_CA.shape

(909, 1)

In [73]:
JCI_AND_QR_P_CA = JCI_P_CA[JCI_P_CA['IDNPROCEEDING'].isin(QR_P_CA['IDNPROCEEDING'])]

In [74]:
# this leads to the same result
JCI_AND_QR_P_CA.shape

(493, 1)

In [75]:
# I want to keep the JCI_P_CA filter, but exclude any situations where a JCI started in CA but was assigned a QR in a different state

In [76]:
QR_P_NOT_CA = schedule[(schedule['ADJ_RSN']=='61') & (~schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNPROCEEDING'].drop_duplicates().to_frame()

In [77]:
JCI_P_CA_CLEAN = JCI_P_CA[~JCI_P_CA.IDNPROCEEDING.isin(QR_P_NOT_CA.IDNPROCEEDING)]

In [78]:
# there are no such cases

print(f'JCI_P_CA: {JCI_P_CA.shape[0]} JCI_P_CA_CLEAN: {JCI_P_CA_CLEAN.shape[0]}')

JCI_P_CA: 1549 JCI_P_CA_CLEAN: 1549


In [79]:
# these are my final 4 filters, 2 for proceedings and 2 for cases
# i can ensure the following

# 1) QR appointed after a JCI (excluding QRs w/ missing JCIs
# 2) QR appointed in the same proceeding as JCI
# 3) JCI and QR appointed in the same state (CA)

# selecting proceedings

JPCA = schedule[(schedule['ADJ_RSN']=='62') & (schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNPROCEEDING'].drop_duplicates().to_frame()
QPCA = schedule[(schedule['ADJ_RSN']=='61') & (schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNPROCEEDING'].drop_duplicates().to_frame()
QPCA = JPCA[JPCA['IDNPROCEEDING'].isin(QPCA['IDNPROCEEDING'])]

# selecting cases 

JCCA = schedule[(schedule['ADJ_RSN']=='62') & (schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNCASE'].drop_duplicates().to_frame()
QCCA = schedule[(schedule['ADJ_RSN']=='61') & (schedule['BASE_CITY_CODE'].isin(CA.BASE_CITY_CODE))]['IDNCASE'].drop_duplicates().to_frame()
QCCA = JCCA[JCCA['IDNCASE'].isin(QCCA['IDNCASE'])]



In [80]:
CASES = {'JCCA':JCCA, 'QCCA':QCCA} 

PROCEEDINGS = {'JPCA':JPCA, 'QPCA':QPCA}

for name, df in CASES.items():
    print(f"Case Category: {name}, Number of cases: {df.shape[0]}")

for name, df in PROCEEDINGS.items():
    print(f"Proceeding Category: {name}, Number of proceedings: {df.shape[0]}")

Case Category: JCCA, Number of cases: 1510
Case Category: QCCA, Number of cases: 539
Proceeding Category: JPCA, Number of proceedings: 1549
Proceeding Category: QPCA, Number of proceedings: 493


In [81]:
proceedings[proceedings['IDNPROCEEDING'].isin(JPCA['IDNPROCEEDING'])].CASE_TYPE.value_counts().head()

RMV    1410
WHO     114
AOC      13
DEP       5
RFR       4
Name: CASE_TYPE, dtype: int64

In [82]:
proceedings[proceedings['IDNPROCEEDING'].isin(JPCA['IDNPROCEEDING'])].NAT.value_counts().head()

MX    849
ES    147
GT    105
HO     98
IN     31
Name: NAT, dtype: int64

In [83]:
proceedings[proceedings['IDNPROCEEDING'].isin(JPCA['IDNPROCEEDING'])].LANG.value_counts().head()

SP     933
ENG    440
PUN     24
CRE     13
MAM     12
Name: LANG, dtype: int64